In [1]:
import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [1]:
model = tf.keras.models.load_model("mask.h5")

NameError: name 'tf' is not defined

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center=True, 
    featurewise_std_normalization=True,
)
train_generator = datagen.flow_from_directory("dataset/train")

In [5]:
classmap = {0: "Mask", 1: "No Mask"}

In [9]:
cap = cv2.VideoCapture("maskvideo.mp4")
(h, w) = (None, None)
detector = cv2.dnn.readNetFromCaffe(
    "Resnet_SSD_deploy.prototxt.txt", "Res10_300x300_SSD_iter_140000.caffemodel")

In [10]:
while True:
    ret, frame = cap.read()
    if h is None:
        (h, w) = frame.shape[:2]
    if ret:
        # gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300),
	(104.0, 177.0, 123.0))
        detector.setInput(blob)
        detections = detector.forward()
        for i in range(0, detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.2:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")
                (startX, startY) = (max(0, startX), max(0, startY))
                (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
                cv2.rectangle(frame, (startX, startY), (endX, endY), (255,0,0), 2)
            

                y = 224 - (startY - endY)
                x = 224 - (startX - endX)


                roi = frame[(startY-y//2):endY+y//2,startX-x//2:endX+x//2]
                
                img = cv2.resize(roi, (224, 224))
                img = cv2.flip(img, 1)
                img = img[np.newaxis,  :, :, ::-1]
                img = datagen.flow(img)
                pred = model.predict(img).ravel()[0]
                predclass = classmap[int(pred>0.5)]

                cv2.putText(frame, predclass, (startX, startY), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)
            cv2.imshow("Detector", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()
#kill the application
os.kill(os.getpid(), signal.SIGTERM)

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-h4wtvo23\opencv\modules\imgproc\src\resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
